In [1]:
#snowpat docs - https://patrick.leibersperger.gitlab-pages.wsl.ch/snowpat/indexspr/#usage
#https://patrick.leibersperger.gitlab-pages.wsl.ch/snowpat/api_reference/snowpat/

import snowpat.snowpackreader as spr


In [2]:
#Read snowpack pro data from path
pro_path = "/Users/travismorrison/Documents/GitHub/UAC-Snowpack/Layer_tracking/ATH201_10_05_2024_03_27_2025.pro"
pro = spr.readPRO(pro_path)


In [3]:
# all available dates
dates = pro.get_all_dates()

# will only return data above the ground after this
pro.discard_below_ground(True)

# get a Snowpack object (internal data class for Profiles) on a specific date
profile = pro.get_profile_on(dates[0])
profile2 = pro.get_profile_nr(12000) # easy acces to the xth profile

# convert it to a dataframe with minimum stability and surface hoar as metadata
# column names will be data codes, except for "0500"= height (layer boundaries)-> 2 columns: layer middle and layer thickness
profile2.toDf().head()


,layer middle,layer thickness,0502,0503,0504,0505,0506,0508,0509,0510,...,0532,0533,0534,0535,0601,0602,0603,0604,0605,0606
0,0.905,0.905,431.8,-0.03,1.0,111.55,0.0,0.0,1.00,3.3,...,1.27,1.23,-4.0,1.12,3.78,0.04,0.0,3.23,-384311.8,-999.00
1,3.050,1.240,425.5,-0.10,5.0,111.54,0.0,0.0,1.00,3.3,...,1.25,1.21,-4.0,1.17,3.66,0.06,0.0,3.21,-364497.9,0.23
2,5.170,0.880,418.4,-0.17,11.0,110.97,0.0,0.0,0.83,3.2,...,1.27,1.23,-4.0,1.12,3.53,0.04,0.0,3.23,-341933.7,0.25
3,6.605,0.555,423.2,-0.21,15.0,110.97,0.0,0.0,1.00,3.3,...,1.31,1.27,-4.0,1.18,3.62,0.30,1.0,3.27,-358859.3,-0.09
4,8.360,1.200,343.1,-0.28,17.0,110.96,0.0,0.0,1.00,3.0,...,1.80,1.74,-5.0,1.48,4.00,0.09,2.0,2.74,-232458.3,-0.07


In [4]:
wl = profile2.weak_layer
sh = profile2.surface_hoar

# There is help, to deal with the DataCodes:
# per default, the names are as in the .pro Header (without units)
pro.update_name_of_code("0503", "Snow Density")
density_code = pro.name_to_code("Snow Density")
                                
#convert to a dataframe, that has wl and sh integrated into the table
#profile2.toDf(integrate=True).head()

# May want to consider re-riting in pyspark
profile2.toDf(CodesToName=pro.DataCodes).head()

,layer middle,layer thickness,element density,Snow Density,element ID,element age,liquid water content by volume,dendricity,sphericity,coordination number,...,natural stability index Sn38,stability index Sk38,0534,optical equivalent grain size,snow shear strength,grain size difference,hardness difference,ssi,inverse texture index ITI,critical cut length
0,0.905,0.905,431.8,-0.03,1.0,111.55,0.0,0.0,1.00,3.3,...,1.27,1.23,-4.0,1.12,3.78,0.04,0.0,3.23,-384311.8,-999.00
1,3.050,1.240,425.5,-0.10,5.0,111.54,0.0,0.0,1.00,3.3,...,1.25,1.21,-4.0,1.17,3.66,0.06,0.0,3.21,-364497.9,0.23
2,5.170,0.880,418.4,-0.17,11.0,110.97,0.0,0.0,0.83,3.2,...,1.27,1.23,-4.0,1.12,3.53,0.04,0.0,3.23,-341933.7,0.25
3,6.605,0.555,423.2,-0.21,15.0,110.97,0.0,0.0,1.00,3.3,...,1.31,1.27,-4.0,1.18,3.62,0.30,1.0,3.27,-358859.3,-0.09
4,8.360,1.200,343.1,-0.28,17.0,110.96,0.0,0.0,1.00,3.0,...,1.80,1.74,-5.0,1.48,4.00,0.09,2.0,2.74,-232458.3,-0.07


In [11]:
def parse_gtype(grain_code):
    """_summary_

    Args:
        grain_code (_type_): _description_

    Returns:
        _type_: _description_
    """
    try:
        primary_gtype = grain_code // 100
        secondary_gtype =  (grain_code % 100) // 10
        tertiary_gtype = grain_code % 10
    except:
         print("grain code parser failed")
         
    return primary_gtype,secondary_gtype, tertiary_gtype


def identify_surface_hoar(profile):
    """_summary_

    Args:
        profile (dataframe): snowpack profile 

    Returns:
       sh_likelihood (boolean): was surface hoar detected at the surface of the profile? 0 not 
    """
    #Swiss Code F1F2F3 Surface Hoar Code is 6, for more information on data formats see : https://snowpack.slf.ch/doc-release/html/snowpackio.html
    #initialize boolean
    sh_likelihood = 0
    try:
        primary_gtype,secondary_gtype, tertiary_gtype = parse_gtype(profile['0513'].iloc[-1])

        #type 4 = SH, type 5 is Depth hoar, type 6 is facets
        if primary_gtype == 4 or primary_gtype == 5 or primary_gtype == 6:
            sh_likelihood = 1
        elif secondary_gtype == 4 or secondary_gtype == 5 or secondary_gtype == 6:
            sh_likelihood = 2
        elif tertiary_gtype == 4 or tertiary_gtype == 5 or tertiary_gtype == 6:
            sh_likelihood = 3
        else:
            sh_likelihood = 0
    except:
        print("surface hoar detection failed")

    return sh_likelihood


In [12]:
profil_end = pro.get_profile_on(dates[10990]) #corrsponds to Jan 27 2025, a period with surface hoar formation 
profile_df = profil_end.toDf()

print(identify_surface_hoar(profile_df))

2


In [ ]:
def identify_near_surface_facets(profile,surface_depth = 50):
    """_summary_

    Args:
        profile (dataframe): snowpack profile 

    Returns:
       sh_likelihood (boolean): was surface hoar detected at the surface of the profile? 0 not 
    """
    #Swiss Code F1F2F3 Surface Hoar Code is 6, for more information on data formats see : https://snowpack.slf.ch/doc-release/html/snowpackio.html
    #initialize boolean
    sh_likelihood = 0
    try:
        search_depth = 0
        layer = len(profile) - 2 #start from second layer since first layer will be caught by SH detector

        #start search from top of snowpack
        while(search_depth<surface_depth):
            search_depth = search_depth + profile['layer thickness'].iloc[layer]
            gtype = profile['0513'].iloc[layer] #grab the variable at layer ii
            #type 4 = SH, type 5 is Depth hoar, type 6 is facets
            if primary_gtype == 4 or primary_gtype == 5 or primary_gtype == 6:
                sh_likelihood = 1
            elif secondary_gtype == 4 or secondary_gtype == 5 or secondary_gtype == 6:
                sh_likelihood = 2
            elif tertiary_gtype == 4 or tertiary_gtype == 5 or tertiary_gtype == 6:
                sh_likelihood = 3
            else:
                sh_likelihood = 0

    except:
        print("near surface facet search fail")

    return sh_likelihood


In [ ]:

def grain_formation_date(grain_type):

    return date, depth